# 타이타닉 생존자 예측
- 2)데이터 확인
- 3)결손치 처리
- 4)문자열 처리
- 5)Survived 속성을 y로 나머지 피쳐를 X로 만듦
- 6)학습/테스트 데이터 세트 분리
- 7)적용할 알고리즘 선정:
    - 결정 트리 – DecisionTreeClassifier
- 8)학습, 예측 및 평가
- 9)교차 검증
- 10) GridSearchCV를 통해 최적 파라미터 도출

In [1]:
import numpy as np
import pandas as pd

### 2) 데이터 확인

In [2]:
titanic_df = pd.read_csv('../00.data/titanic/train.csv')
titanic_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### 3)결손치 처리

In [4]:
# nan값확인
titanic_df.isnull().values.any()

True

In [5]:
# nan값 변경

# Age는 nan 값을 평균나이로 변경 
titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True) 

# 나머지는 n으로 변경 
titanic_df['Cabin'].fillna('N',inplace = True)
titanic_df['Embarked'].fillna('N',inplace = True)
print('데이터 셋 Null 값 개수',titanic_df.isnull().sum().sum())

데이터 셋 Null 값 개수 0


In [6]:
# str형의 1글자만 앞에서 가져온다
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df['Cabin'].head(3)

0    N
1    C
2    N
Name: Cabin, dtype: object

In [7]:
titanic_df.tail(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.45,N,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.00,C,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.000000,0,0,370376,7.75,N,Q


### 4) 문자열 처리

In [8]:
from sklearn.preprocessing import LabelEncoder

# sex 칼럼의 특정 값들을 문자열 변경할때
def get_sex(data):

    en = LabelEncoder()
    en.fit(['male','female'])
    data["Sex"] = en.transform(data["Sex"])
    return data

titanic_df = get_sex(titanic_df)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,N,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,N,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,N,S


In [9]:
def encode_features(dataDF):
    features = ['Cabin','Embarked']
    for feature in features:
        encoder = LabelEncoder()
        enc = encoder.fit(dataDF[feature])
        dataDF[feature] = enc.transform(dataDF[feature])
    return dataDF

titanic_df = encode_features(titanic_df)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,7,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,2,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,7,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,2,3
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,7,3


In [10]:
# 불필요한 속성 삭제
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1,inplace=True)
    return df

### 5) Survived 속성을 y로 나머지 피쳐를 X로 만듦

In [11]:
# 피처 데이터 세트와 레이블 데이터 세트 추출.
y_titanic_df = titanic_df['Survived'] # y_titenic_df에 정답이 되는 생존 여부를 저장
X_titanic_df = titanic_df.drop('Survived',axis=1) # x_titanic_df는 survived를 없앤다.

In [12]:
X_titanic_df = drop_features(X_titanic_df)

### 6) 학습/테스트 데이터 세트 분리

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(
    X_titanic_df, y_titanic_df, test_size = 0.2, random_state=11
)

### 7) 적용할 알고리즘 선정:
- 결정 트리 – DecisionTreeClassifier

In [14]:
from sklearn.tree import DecisionTreeClassifier # 회기/ 분류
from sklearn.metrics import accuracy_score

In [15]:
# 객체 생성 --> 모델 생성
dt_clf = DecisionTreeClassifier(random_state=11)

#### 머신러닝 알고리즘을 적용해 학습

In [16]:
# 학습 수행
dt_clf.fit(X_train, y_train)

# 예측값
dt_pred = dt_clf.predict(X_test)
print(f'정확도: {accuracy_score(y_test, dt_pred ):.4f}')

정확도: 0.7877


### 9) 교차 검증

### GridSearchCV를 통하여 결정트리 모델의 최적 파라미터 도출

In [20]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth':[2, 3, 4], 
    'min_samples_split':[2, 3]
}

X_train, X_test, y_train, y_test= train_test_split(
    X_titanic_df, y_titanic_df, test_size = 0.2, random_state=11
)

dtree = DecisionTreeClassifier()
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, scoring='accuracy', refit=True)
grid_dtree.fit(X_train, y_train)

print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCv 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCv 최고 정확도: 0.7936


In [21]:
estimator = grid_dtree.best_estimator_

In [22]:
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도: 0.8715


### 결정 트리 모델을 cross_val_score 함수를 이용하여 교차 검증

In [19]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_titanic_df, y_titanic_df, cv=5)
for iter_count, accuracy in enumerate(scores):
    print(f"교차 검증 {iter_count} 정확도: {accuracy:.4f}")

print(f"평균 정확도: {np.mean(scores):.4f}")

교차 검증 0 정확도: 0.7430
교차 검증 1 정확도: 0.7753
교차 검증 2 정확도: 0.7921
교차 검증 3 정확도: 0.7865
교차 검증 4 정확도: 0.8427
평균 정확도: 0.7879


### Stratified K-Fold 방식으로 교차 검증하는 함수

In [17]:
from sklearn.model_selection import StratifiedKFold

def exec_skfold(clf, folds=5):
    # 폴드 세트를 5개인 KFold객체를 생성, 폴드 수만큼 예측결과 저장을 위한  리스트 객체 생성.
    skfold = StratifiedKFold(n_splits=folds)
    scores = []
    
    # KFold 교차 검증 수행. 
    for iter_count, (train_index, test_index) in enumerate(skfold.split(X_titanic_df, y_titanic_df)):
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        
        # Classifier 학습, 예측, 정확도 계산 
        clf.fit(X_train, y_train) 
        pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        scores.append(accuracy)
        print(f"교차 검증 {iter_count} 정확도: {accuracy:.4f}")     
    
    # 5개 fold에서의 평균 정확도 계산. 
    mean_score = np.mean(scores)
    print(f"평균 정확도: {mean_score:.4f}")

In [18]:
# DecisionTreeClassifier 모델 교차 검증
exec_skfold(dt_clf, folds=5)

교차 검증 0 정확도: 0.7430
교차 검증 1 정확도: 0.7753
교차 검증 2 정확도: 0.7921
교차 검증 3 정확도: 0.7865
교차 검증 4 정확도: 0.8427
평균 정확도: 0.7879
